# [ Business Problem ]

## 조기 퇴사하는 사람들은 어떤 특성을 가지고 있는가?
## 조기 퇴사를 막기 위해 어떤 사람들을 타겟팅해야 하는가?

#  

# [ Data ]

## 조기 퇴사 여부가 표시된, 인사 고과 데이터
## https://www.kaggle.com/ludobenistant/hr-analytics

#  

# [ Chap1. 데이터 탐색 ]

## 분석의 첫 단계
## 모델링에 앞서 데이터에 대한 감을 잡는데 중요하다
## EDA(Exploratory Data Analysis)라고도 하며, 여러가지 시각화를 통해 수행된다

#   

### Pandas 라이브러리 import

In [ ]:
import pandas as pd

### 데이터 로드

In [ ]:
hr_data = pd.read_csv('data/HR.csv')

### 데이터에 존재하는 변수 확인

In [ ]:
hr_data.columns

### 각 변수의 타입 확인

In [ ]:
hr_data.dtypes

#### float, int = 수치형 변수, object, str = 문자형 변수

### 데이터 테이블 형태 확인

In [ ]:
hr_data.head()

### 각 변수의 값을 살펴보기

In [ ]:
# hr_data.satisfaction_level
hr_data['satisfaction_level']

### 변수가 가질 수 있는 값 확인

In [ ]:
hr_data['sales'].unique()

In [ ]:
hr_data['number_project'].unique()

### 조건에 맞는 데이터 살펴보기

In [ ]:
hr_data.loc[hr_data['sales'] == 'IT']

In [ ]:
hr_data.loc[hr_data['last_evaluation'] > 0.95]

### 각 변수 별 요약통계량

In [ ]:
hr_data.describe()

#### describe함수는 수치형 변수와 범주형 변수가 같이 있는 경우, 수치형 변수에 대한 요약만 제공한다
#### 범주형 변수에 대한 요약 통계는 범주형 변수에 대해서만 describe 함수를 사용하면 확인할 수 있음

In [ ]:
hr_data[['sales','salary']].describe()

### 요약통계량도 데이터에 대한 좋은 정보지만, 데이터 시각화를 통해 더 효과적으로 데이터를 살펴볼 수 있음

### 시각화를 위한 라이브러리인 matplotlib, seaborn를 import

#### pip install seaborn

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

#### Jupyter notebook에서 도표 출력

In [ ]:
% matplotlib inline

### 수치형 변수는 histogram으로, 범주형 변수는 barplot으로 분포를 시각화

### pyplot의 hist() 함수로 histogram을 그릴 수 있음

#### satisfaction_level 변수의 분포 시각화

In [ ]:
plt.figure(figsize=(10,2))
plt.hist(hr_data['satisfaction_level'])

#### bins 옵션을 통해 히스토그램 막대의 개수를 조절할 수 있음

In [ ]:
plt.figure(figsize=(10,2))
plt.hist(hr_data['satisfaction_level'], bins=5)

#### average_montly_hours 변수 분포 시각화

In [ ]:
plt.figure(figsize=(10,2))
plt.hist(hr_data['average_montly_hours'], bins=5)

### 조기 퇴사 한 경우와, 그렇지 않은 경우 사이에 만족도, 월 근무 시간 분포에 차이가 있을까?

#### 만족도의 차이

In [ ]:
plt.figure(figsize=(10,2))
plt.hist(hr_data.loc[hr_data['left']==0,'satisfaction_level'], bins=5, alpha=0.5)
plt.hist(hr_data.loc[hr_data['left']==1,'satisfaction_level'], bins=5, alpha=0.5)

#### 근무 시간의 차이

In [ ]:
plt.figure(figsize=(10,2))
plt.hist(hr_data.loc[hr_data['left']==0,'average_montly_hours'], bins=5, alpha=0.5)
plt.hist(hr_data.loc[hr_data['left']==1,'average_montly_hours'], bins=5, alpha=0.5)

#### 범주형 변수의 분포 시각화. Barplot은 seaborn 라이브러리의 countplot() 함수로 그릴 수 있음

#### sales 변수 값(부서) 별 인원 수

In [ ]:
# hr_data.sales[:100]

In [ ]:
plt.figure(figsize=(15,2))
sns.countplot(x='sales', data=hr_data)

#### 인원 수가 sales 쪽이 가장 많음을 알 수 있다

### 부서에 따라 조기퇴사비율을 살펴보자

In [ ]:
plt.figure(figsize=(15,2))
sns.countplot(x='sales', hue='left', data=hr_data)

### Salary 변수의 분포를 살펴보자

In [ ]:
# hr_data.salary[:100]

In [ ]:
plt.figure(figsize=(10,2))
sns.countplot(x='salary', data=hr_data)

### Salary 수준 별로 조기 퇴사 비율을 살펴보자

In [ ]:
plt.figure(figsize=(10,2))
sns.countplot(x='salary', hue='left', data=hr_data)

## [ 실습 ]

### 1) last_evaluation, number_project, time_spend_company 변수에 대해
### 조기 퇴사 여부에 따라 두 개의 histogram을 그려보자. 조기 퇴사 여부에 따라 어떤 차이가 있는가?

### 2) Work_accident, promotion_last_5years 변수에 대해
### 조기 퇴사 여부에 따른 barplot을 그려보자. 조기 퇴사 여부에 따라 어떤 차이가 있는가?

#  

### 변수 간 상관관계

In [ ]:
hr_data.corr()

In [ ]:
correlation = hr_data.corr()
plt.figure(figsize=(10,3))
sns.heatmap(correlation, annot=True)
plt.title('Correlation between different fearures')

#### 관심있는 변수인 Left(조기 퇴사)와 Satisfaction_level과 음의 상관관계
#### time_spend_company 와는 양의 상관관계

#   

#   

# [ Chap2. 모델링 ] 

### 데이터 탐색, 시각화로도 많은 것을 알 수 있다, 하지만
### 1) 시각화만으로 분석하기에는 살펴보아야 할 것이 너무 많다
### 2) 여러 변수가 관계된 경우 2차원 시각화로는 데이터에 나타나는 패턴을 관찰하기 힘들다
###  
### 모델링을 통해
### 1) 데이터에 나타나는 패턴을 정량화 할 수 있고
### 2) 그러한 패턴이 일반화 가능한 패턴인지 검증할 수 있으며
### 3) 다변수적인 분석이 가능하다

### 모델은 분석가의 길잡이 역할을 해 줄 수 있음

##  

### 데이터 전처리

### 모델은 숫자로 된 데이터만을 처리할 수 있음. 문자형 변수에 대한 처리가 필요

In [ ]:
hr_data.head()

In [ ]:
# hr_data[['promotion_last_5years', 'Work_accident', 'sales', 'salary']]

In [ ]:
# pd.get_dummies(hr_data[['promotion_last_5years', 'Work_accident', 'sales', 'salary']])

In [ ]:
numerical_data = hr_data[['satisfaction_level', 'last_evaluation', 'number_project',
                          'average_montly_hours', 'time_spend_company']]

In [ ]:
categorical_data = hr_data[['promotion_last_5years', 'Work_accident', 'sales', 'salary']]

In [ ]:
class_variable = hr_data['left']

In [ ]:
hr_data_dummy_coded = pd.concat([numerical_data, pd.get_dummies(categorical_data), class_variable], axis=1)

In [ ]:
# hr_data_dummy_coded

### 입력 변수, 반응 변수 분리

In [ ]:
X = hr_data_dummy_coded.drop('left', axis=1)

In [ ]:
y = hr_data_dummy_coded['left']

In [ ]:
print(X.shape, y.shape)

### 학습 데이터 / 검증 데이터 분할

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_val.shape, y_val.shape)

### 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr.predict(X_train)

### 모델의 정확도 평가

In [ ]:
from sklearn.metrics import accuracy_score

#### 학습 데이터에서의 성능

In [ ]:
accuracy_score(y_train, lr.predict(X_train))

#### 검증 데이터에서의 성능

In [ ]:
accuracy_score(y_val, lr.predict(X_val))

#### 얼마나 정확한건가?

In [ ]:
import numpy as np

#### 클래스 1의 비율을 계산

In [ ]:
np.mean(y)

In [ ]:
1 - np.mean(y)

In [ ]:
plt.figure(figsize=(10,2))
sns.countplot(x='left',data=hr_data)

#### Random guess의 성능?

#### Accuracy의 절대적인 값은 높지만, 클래스 비율을 고려하면 높다고 할 수 없음

#### 모델의 성능을 평가하는 다른 척도?

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_val, lr.predict(X_val))

#### 행이 true label, 열이 predicted label

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
precision_score(y_val, lr.predict(X_val))

In [ ]:
recall_score(y_val, lr.predict(X_val))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_val, lr.predict(X_val))

#### 선형으로 잘 분류되지 않으니, 비선형 분류 모델을 고려해보자

### 의사결정나무

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=8)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
accuracy_score(y_train, dt.predict(X_train))

In [ ]:
accuracy_score(y_val, dt.predict(X_val))

In [ ]:
precision_score(y_val, dt.predict(X_val))

In [ ]:
recall_score(y_val, dt.predict(X_val))

In [ ]:
f1_score(y_val, dt.predict(X_val))

### 비선형 모델이 좋은 성능을 냄
### 입력 변수와 반응 변수 간에는 비선형적 관계가 있다고 할 수 있다

##   

### 학습된 의사결정나무를 시각화 해보자

#### 트리 시각화를 위해 graphviz, pydotplus 라이브러리 필요

    (1)Graphviz(http://www.graphviz.org/) - graphviz-2.38.msi로 설치

    (2)환경변수 경로설정(C:\Program Files (x86)\Graphviz2.38\bin)

    (3)[제어판 -> 시스템 및 보안 -> 시스템 -> (좌측) 고급 시스템 설정 -> 환경변수 -> Path에 위 경로 추가]

    (4)conda install graphviz

    (5)pip install pydotplus

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus as pydot

In [ ]:
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data)
graph = pydot.graph_from_dot_data(dot_data.getvalue())

In [ ]:
graph.write_jpeg('tree.jpeg')

In [ ]:
Image(filename='tree.jpeg')

### 몇 가지 옵션을 통해 각 노드의 의미를 알아보기 쉽게 만들 수 있음

In [ ]:
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data, feature_names=X.columns, impurity=False, proportion=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())

In [ ]:
graph.write_jpeg('tree2.jpeg')
Image(filename='tree2.jpeg')

### 그러나 규칙을 해석하기에는 트리 구조가 너무 복잡하다

### 모델링 결과를 이해하기 위해서는, 트리 모델이 정확하면서도 간결(복잡도가 낮아야 함)하여야 함

##   

### 트리의 복잡도 별로, 학습/검증 데이터에서의 정확도가 어떻게 변화하는지 관찰해보자

#### 트리 모델의 복잡도는 트리의 깊이로 조절할 수 있음

In [ ]:
from sklearn.learning_curve import validation_curve

In [ ]:
param_range = range(1,15)
train_scores, test_scores = validation_curve(DecisionTreeClassifier(), 
                                             X, y, 
                                             param_name="max_depth",
                                             param_range=param_range,
                                             cv=10,
                                             scoring="accuracy")
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
train_scores_mean

In [ ]:
test_scores_mean

### 트리의 깊이 별로 구한 10 Fold Cross Validation Accuracy를 시각화해보자

In [ ]:
plt.figure(figsize=(10,2))

plt.xlabel("Depth of Tree")
plt.ylabel("Accuracy")

plt.plot(param_range, train_scores_mean, label="Training Acc", color="r")
plt.plot(param_range, test_scores_mean, label="Validation Acc", color="g")
plt.legend(loc='lower right')

plt.show()

### Error (1-Accuracy)를 y축으로 하여 표현해보자

In [ ]:
plt.figure(figsize=(10,2))

plt.xlabel("Depth of Tree")
plt.ylabel("Accuracy")

plt.plot(param_range, 1 - train_scores_mean, label="Training Error", color="r")
plt.plot(param_range, 1 - test_scores_mean, label="Validation Error", color="g")
plt.legend()

plt.show()

### 모델의 정확도와 해석력을 종합적으로 고려하여, 깊이가 3인 트리를 선택

In [ ]:
best_tree = DecisionTreeClassifier(max_depth=3)

In [ ]:
best_tree.fit(X,y)

In [ ]:
accuracy_score(y, best_tree.predict(X))

In [ ]:
confusion_matrix(y, best_tree.predict(X))

In [ ]:
print('Precision Score: ', precision_score(y, best_tree.predict(X)))
print('Recall Score: ', recall_score(y, best_tree.predict(X)))

In [ ]:
dot_data = StringIO()
export_graphviz(best_tree, out_file=dot_data, feature_names=X.columns, impurity=False, proportion=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph.write_jpeg('best_tree.jpeg')
Image(filename='best_tree.jpeg')

#   

##  [실습]

### 트리의 옵션을 변경하여, 다른 형태의 트리를 그릴 수도 있다

#### tip) 옵션이 궁금한 곳에 커서를 올려놓고, shift + tap을 한 번/두 번 눌러보자

## 각 Leaf 노드에 속하는 인스턴스의 수의 최소 기준(min_samples_leaf)을 조정해보자

#   

#   

# [ Chap3. Insight ] - 어떤 사람들이 조기 퇴사하는가?

### 조기 퇴사를 예측하는 규칙들을 정리해보자

In [ ]:
cond1 = X.satisfaction_level <= 0.465
cond2 = X.number_project <= 2.5
cond3 = X.time_spend_company <= 4.5
cond4 = X.last_evaluation <= 0.575 
cond5 = X.satisfaction_level <= 0.115
cond6 = X.average_montly_hours <= 290.5
cond7 = X.last_evaluation <= 0.805

In [ ]:
left_group1 = cond1 & cond2 & cond4
left_group2 = cond1 & np.bitwise_not(cond2) & cond5
left_group3 = np.bitwise_not(cond1) & cond3 & np.bitwise_not(cond6)
left_group4 = np.bitwise_not(cond1) & np.bitwise_not(cond3) & np.bitwise_not(cond7)

<h4>> 그룹1: SATISFACTION_LEVEL <= 0.465, NUMBER_PROJECT <= 2.5, LAST_EVALUATION <= 0.575 </h4>
<h4>> 그룹2: SATISFACTION_LEVEL <= 0.115, NUMBER_PROJECT > 2.5</h4>
<h4>> 그룹3: SATISFACTION_LEVEL > 0.465, TIME_SPEND_COMPANY <= 4.5, AVERAGE_MONTLY_HOURS > 290.5 </h4>
<h4>> 그룹4: SATISFACTION_LEVEL > 0.465, TIME_SPEND_COMPANY > 4.5, LAST_EVALUATION > 0.805 </h4>

## 각 그룹 별 크기

In [ ]:
len(X.loc[left_group1])

In [ ]:
len(X.loc[left_group2])

In [ ]:
len(X.loc[left_group3])

In [ ]:
len(X.loc[left_group4])

In [ ]:
len(X.loc[y==1])

## 각 변수의 중요도는 어떻게 되는가?

In [ ]:
best_tree.feature_importances_

In [ ]:
important_features = dict()
for feature_name, importance in zip(X.columns, best_tree.feature_importances_):
    if importance != 0:
        important_features[feature_name] = importance

### Feature를 중요도 순으로 정렬

In [ ]:
tree_feature_importance = sorted(important_features.items(), key=lambda x: -x[1])

In [ ]:
tree_features = []
tree_feature_importances = []
for feature, importance in tree_feature_importance:
    tree_features.append(feature)
    tree_feature_importances.append(importance)
    print(feature, importance)

In [ ]:
plt.figure(figsize=(10,2))
sns.barplot(tree_features, tree_feature_importances)

### 조기 퇴사할 확률?

In [ ]:
best_tree.predict_proba(X)

In [ ]:
left_probs = []
for prob in best_tree.predict_proba(X):
    left_probs.append(prob[1])

In [ ]:
# left_probs

In [ ]:
X['left_prob_tree'] = left_probs

In [ ]:
# X

### 데이터 시각화를 통해 중요 변수와 조기 퇴사 확률과의 관계를 살펴보자

In [ ]:
plt.figure(figsize=(5,3))
plt.scatter(x=X.satisfaction_level, y=X.last_evaluation, c=X.left_prob_tree)
plt.xlabel('Satisfaction Level')
plt.ylabel('Last Evaluation')

In [ ]:
plt.figure(figsize=(5,3))
plt.scatter(x=X.satisfaction_level, y=X.average_montly_hours, c=X.left_prob_tree)
plt.xlabel('Satisfaction Level')
plt.ylabel('Average Montly Hours')

In [ ]:
plt.figure(figsize=(5,3))
plt.scatter(x=X.average_montly_hours, y=X.last_evaluation, c=X.left_prob_tree)
plt.xlabel('Average Montly Hours')
plt.ylabel('Last Evaluation')

### 세 가지 변수를 한 번에 나타내보자

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

In [ ]:
fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=X.satisfaction_level, ys=X.last_evaluation, zs=X.average_montly_hours, c=X.left_prob_tree)

ax.set_xlabel('Satisfaction Level')
ax.set_ylabel('Last Evaluation')
ax.set_zlabel('Average Montly Hours')

plt.show()

#### 크게 세 그룹 정도가 있음을 알 수 있다

### 또 다른 중요변수인 time_spend_company, number_project와 조기 퇴사 확률 간의 관계 시각화

#### histogram과의 차이? y축에 유의(histogram은 count. 아래의 plot은 각 변수 수준에서 조기 퇴사 확률의 평균)

In [ ]:
plt.figure(figsize=(10,2))
sns.barplot(x='time_spend_company', y='left_prob_tree', data=X)

In [ ]:
plt.figure(figsize=(10,2))
sns.barplot(x='number_project', y='left_prob_tree', data=X)

### 각 인스턴스가 어느 left group에 속하는지 레이블 부여

In [ ]:
hr_data['left_group'] = 'Not Leaving' # Default

In [ ]:
hr_data.loc[left_group1, 'left_group'] = 'A'
hr_data.loc[left_group2, 'left_group'] = 'B'
hr_data.loc[left_group3, 'left_group'] = 'C'
hr_data.loc[left_group4, 'left_group'] = 'D'

### 조기 퇴사 유형 별 비율을 시각화

In [ ]:
plt.figure(figsize=(10,3))
sns.countplot(x='left_group', data=hr_data)

### 부서 별로 조기 퇴사 유형 분포를 살펴보자

In [ ]:
plt.figure(figsize=(10,3))
sns.countplot(x='sales', hue='left_group', data=hr_data)
plt.legend(loc='upper right')

#### Not Leaving이 너무 많으니, 제외하고 보자

In [ ]:
plt.figure(figsize=(10,3))
sns.countplot(x='sales', hue='left_group', data=hr_data.loc[hr_data['left_group'] != 'Not Leaving'])
plt.legend(loc='upper right')

#### 부서 별로 퇴사 유형 분포에 차이가 있음을 알 수 있다

#   

#   

# [ Chap4. Action ] - 어떤 사람들에게 조치를 취해야 하는가?

### 클래스 = 0 인 사원 중, 예측 값이 높은 인원들 제시

In [ ]:
hr_data['left_prob'] = left_probs

In [ ]:
hr_data.loc[hr_data['left']==0].sort_values(by='left_prob', ascending=False)

### 퇴사 확률에 따라, 인원 수가 얼마나 되는지 살펴보자

In [ ]:
plt.hist(hr_data['left_prob'])

### (퇴사하지 않았지만) 조기 퇴사할 확률이 높은 그룹

In [ ]:
hr_data.loc[np.bitwise_and(hr_data['left']==0, hr_data['left_prob'] >= 0.7)]

### 위 그룹에 대해 별도의 조치가 필요. 그런데 비용이 크다면?

### 과거에 우수한 성과를 보였던 인원들을 우선적으로 타겟팅 하는 것이 효율적일 것

### 조기 퇴사 확률과 과거 성과를 동시에 고려하는, 조치의 우선 순위를 정해보자

In [ ]:
hr_data['priority'] = (hr_data['satisfaction_level'] + hr_data['left_prob']) / 2

In [ ]:
plt.hist(hr_data['priority'])

In [ ]:
target_group = hr_data.loc[np.bitwise_and(hr_data['left']==0, hr_data['priority'] > 0.65)]

In [ ]:
target_group

### 타겟 그룹에서, 퇴사 유형 비율은 어떠한가?

In [ ]:
sns.countplot(x='left_group', data=target_group)

### D와 A 그룹이 타겟 그룹에서 두드러짐

#   

## [실습]

### 우선 순위를 계산 시, 조기 퇴사 확률과 과거 성과 사이에 가중치를 조정해보자
### i.e. 조기 퇴사 확률 : 과거 성과 = 1:2
### 타겟 그룹은 어떻게 변화하는가?

#   

### 비즈니스적 목적성에 맞게 적절한 지표를 잡는 것이 중요함

#   

# [ Q&A ]

#   

### 더 궁금한 것은 taewook@dm.snu.ac.kr 로..